In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
sns.set_theme()

# optimization
from scipy import optimize

import statsmodels.formula.api as smf

from scipy.stats import norm
from scipy.stats import genextreme

%load_ext autoreload
%autoreload 2

import clogit_post as clogit
import estimation as est

Import Data

In [2]:
cars = pd.read_csv('cars.csv')
lbl_vars = pd.read_csv('labels_variables.csv')
lbl_vals = pd.read_csv('labels_values.csv')

# convert from dataframe to dict
lbl_vals = {c: lbl_vals[c].dropna().to_dict() for c in lbl_vals.columns}

lbl_vars.set_index('variable', inplace=True)

Summarise Dataset

In [3]:
lbl_vars.join(cars.mean(numeric_only=True).apply(lambda x: f'{x: .2f}').to_frame('Mean'))

label  \
variable                                                      
ye                         year (=first dimension of panel)   
ma                      market (=second dimension of panel)   
co                   model code (=third dimension of panel)   
zcode     alternative model code (predecessors and succe...   
brd                                              brand code   
type                                name of brand and model   
brand                                         name of brand   
model                                         name of model   
org       origin code (demand side, country with which c...   
loc       location code (production side, country where ...   
cla                                   class or segment code   
home      domestic car dummy (appropriate interaction of...   
frm                                               firm code   
qu                  sales (number of new car registrations)   
cy                  cylinder volume or displacement (in cc)   
hp                                       horsepower (in kW)   
we                                           weight (in kg)   
pl                   places (number, not reliable variable)   
do                    doors (number, not reliable variable)   
le                                           length (in cm)   
wi                                            width (in cm)   
he                                           height (in cm)   
li1       measure 1 for fuel efficiency (liter per km, a...   
li2       measure 2 for fuel efficiency (liter per km, a...   
li3       measure 3 for fuel efficiency (liter per km, a...   
li                average of li1, li2, li3 (used in papers)   
sp                                  maximum speed (km/hour)   
ac        time to acceleration (in seconds from 0 to 100...   
pr         price (in destination currency including V.A.T.)   
princ     =pr/(ngdp/pop): price relative to per capita i...   
eurpr     =pr/avdexr: price in common currency (in SDR t...   
exppr                 =pr/avexr: price in exporter currency   
avexr     av. exchange rate of exporter country (exporte...   
avdexr    av. exchange rate of destination country (dest...   
avcpr          av. consumer price index of exporter country   
avppr          av. producer price index of exporter country   
avdcpr      av. consumer price index of destination country   
avdppr      av. producer price index of destination country   
xexr                                           avdexr/avexr   
tax                                          percentage VAT   
pop                                              population   
ngdp      nominal gross domestic product of destination ...   
rgdp                            real gross domestic product   
engdp     =ngdp/avdexr: nominal gdp in common currency (...   
ergdp                                           =rgdp/avexr   
engdpc    =engdp/pop: nominal gdp per capita in common c...   
ergdpc                                           =ergdp/pop   
s                                market share (qu / qu_tot)   
qu_tot                      total sales in this market-year   
inc                                  avg. income per capita   

                         Mean  
variable                       
ye                      84.50  
ma                       3.00  
co                     207.50  
zcode                  177.76  
brd                     16.79  
type                      NaN  
brand                     NaN  
model                     NaN  
org                      2.72  
loc                      5.17  
cla                      2.30  
home                     0.32  
frm                     14.50  
qu                   35606.68  
cy                    1337.09  
hp                      50.10  
we                     934.49  
pl                       4.88  
do                       3.55  
le                     409.24  
wi                     163.44  
he                     140.46  
li1      

Prepare dataset for clogit analysis

In [4]:
price_var = 'princ'

cars['logp'] = np.log(cars[price_var])

# new variable: price elasticity heterogeneous for home-region 
cars['logp_x_home'] = cars[price_var] * cars['home']



Identify dummies and add columns

In [5]:
categorical_var = 'brand' # name of categorical variable
dummies = pd.get_dummies(cars[categorical_var]) # creates a matrix of dummies for each value of dummyvar
dummies = dummies.astype(int)

x_vars_dummies = list(dummies.columns[1:].values) # omit a reference category, here it is the first (hence columns[1:])

# add dummies to the dataframe 
assert dummies.columns[0] not in cars.columns, f'It looks like you have already added this dummy to the dataframe. Avoid duplicates! '
cars = pd.concat([cars,dummies], axis=1)

Select variables

In [13]:
x_vars = ['logp', 'cy', 'hp', 'we', 'li']
print(f'K = {len(x_vars)} standardized variables selected.')

K = len(x_vars)
N = cars.ma.nunique() * cars.ye.nunique()
J = 40 
x = cars[x_vars].values.reshape((N,J,K))
y = np.log(cars['s'].values.reshape((N,J)))


mean_x = np.mean(x, axis=0, keepdims=True)
std_x = np.std(x, axis=0, keepdims=True)
x = (x - mean_x) / std_x


dummies = ['home'] + x_vars_dummies
K = len(dummies)
print(f'K = {K} dummy variables selected.')

dummy_vars = cars[dummies].values.reshape((N, J, K))
x = np.concatenate([x, dummy_vars], axis=2)

K = len(x_vars) + len(dummies)

print(f'K = {K} total variables selected.')

theta0 = np.ones((K,))
res = est.estimate(clogit.q,theta0=theta0,y=y,x=x)

print(est.print_table(theta_label=(x_vars + dummies),results=res))



K = 5 standardized variables selected.
K = 33 dummy variables selected.
K = 38 total variables selected.
Optimization terminated successfully.
         Current function value: -61116415582.486786
         Iterations: 2
         Function evaluations: 585
         Gradient evaluations: 15
Optimizer succeded after 2 iter. (585 func. evals.). Final criterion: -6.112e+10.
Results
                    theta            se       t
logp         3.269832e+07  9.701128e+10  0.0003
cy           3.951819e+07  3.832081e+11  0.0001
hp           3.991443e+07  3.372316e+11  0.0001
we           3.657077e+07  2.554613e+10  0.0014
li           2.890005e+07  4.242434e+10  0.0007
home         5.477245e+06  4.575820e+10  0.0001
MCC         -6.721552e+03  4.162926e+07 -0.0002
VW          -1.235064e+06  1.427755e+10 -0.0001
alfa romeo   3.822596e+05  1.987645e+09  0.0002
audi        -5.216164e+05  3.403877e+09 -0.0002
citroen      7.177783e+04  6.345473e+08  0.0001
daewoo      -2.071838e+04  2.821634e+08 -0.000

Finding APEs

In [7]:
def marg_effect_discrete(betahat, x0, x1):
    # Calculate the probabilities of x1 and x0, using betahat of the lates model estimation.
    # Then substract the probability vector of x0 from the probability vector of x1.
    prob_x1 = clogit.choice_prob(betahat, x1)
    prob_x0 = clogit.choice_prob(betahat, x0)
    return (prob_x1 - prob_x0).flatten()

x0 = x.copy()
x0[:,:,5] = 0

x1 = x.copy()
x1[:,:,5] = 1

pe = marg_effect_discrete(res['theta'], x0, x1)

ape = np.mean(pe)

print(ape)

0.0


Finding elasticity

In [8]:
thetahat = res['theta']

ccp1 = clogit.choice_prob(thetahat, x)

E_own   = np.zeros((N, J))
E_cross = np.zeros((N, J))
k_price = 0 

for j in range(J):
    # A. copy 
    x2 = x.copy()
    
    # B. increase price just for car j 
    rel_change_x = 1e-2
    x2[:, j, k_price] *= (1.0+rel_change_x)
    
    # C. evaluate CCPs
    ccp2 = clogit.choice_prob(thetahat, x2)
    
    # D. percentage change in CCPs 
    rel_change_y = ccp2 / (ccp1+1e-12) - 1.0 
    
    # E. elasticities 
    elasticity = rel_change_y / rel_change_x 
    
    E_own[:, j] = elasticity[:, j]
    
    k_not_j = [k for k in range(J) if k != j]
    E_cross[:, j] = elasticity[:, k_not_j].mean(axis=1)

print(f'Own-price elasticity:  {np.mean(E_own).round(4)}')
print(f'Cross-price elasticity: {np.mean(E_cross).round(4)}')

i_home = 5

idx1 = x[:, :, i_home]==1
idx0 = x[:, :, i_home]==0 
print(f'Elasticity, home:   {np.mean(E_own[idx1]).round(4)}')
print(f'Elasticity, other: {np.mean(E_own[idx0]).round(4)}')

Own-price elasticity:  -97.5
Cross-price elasticity: -97.5
Elasticity, home:   -96.2592
Elasticity, other: -98.0741
